# Solveur VRPTW avec ALNSCe notebook implémente un algorithme pour résoudre le **Vehicle Routing Problem with Time Windows (VRPTW)** en utilisant une approche hybride métaheuristique.## Composants de l'algorithme:1. **Construction initiale**: Heuristique de Clark-Wright pour créer une solution initiale2. **Amélioration locale**: Variable Neighborhood Descent (VND) avec opérateurs relocate, swap et 2-opt3. **Recherche globale**: Adaptive Large Neighborhood Search (ALNS)## Objectif:Trouver des solutions de haute qualité pour les instances VRPTW de Solomon avec un écart **≤ 5%** par rapport aux meilleures solutions connues.## Contraintes:- Capacité des véhicules- Fenêtres temporelles pour chaque client- Temps de service## Format des données:Instances au format Solomon (.txt)

In [ ]:
# Importation des bibliothèques nécessairesimport numpy as npimport vrplibimport yamlimport randomimport mathimport timeimport osfrom collections import dequefrom copy import deepcopyfrom typing import List, Dict, Tuple, Optionalimport warningsimport matplotlib.pyplot as pltimport pandas as pdwarnings.filterwarnings('ignore')print("Bibliothèques importées avec succès!")

In [ ]:
# Charger la configuration depuis le fichier YAMLwith open('config_ALNS.yaml', 'r', encoding='utf-8') as f:    config = yaml.safe_load(f)print("Configuration chargée:")print(yaml.dump(config, default_flow_style=False, allow_unicode=True))

In [ ]:
# Définir la graine aléatoire pour la reproductibilitérandom.seed(config['general']['random_seed'])np.random.seed(config['general']['random_seed'])print(f"Graine aléatoire définie à: {config['general']['random_seed']}")

## Structures de données et fonctions utilitaires

In [ ]:
class VRPTWSolution:    """Classe pour représenter une solution VRPTW"""        def __init__(self, instance):        self.instance = instance        self.routes = []  # Liste des routes (chaque route est une liste d'indices de clients)        self.cost = float('inf')  # Coût total de la solution        self.is_feasible = True  # La solution respecte-t-elle toutes les contraintes?        self.capacity_violations = 0  # Nombre de violations de capacité        self.time_violations = 0  # Nombre de violations de fenêtres temporelles            def copy(self):        """Créer une copie profonde de la solution"""        new_sol = VRPTWSolution(self.instance)        new_sol.routes = deepcopy(self.routes)        new_sol.cost = self.cost        new_sol.is_feasible = self.is_feasible        new_sol.capacity_violations = self.capacity_violations        new_sol.time_violations = self.time_violations        return new_sol        def get_all_customers(self):        """Obtenir tous les clients dans la solution"""        customers = []        for route in self.routes:            customers.extend(route)        return customers        def remove_empty_routes(self):        """Supprimer les routes vides"""        self.routes = [r for r in self.routes if len(r) > 0]            def __str__(self):        return f"Solution: {len(self.routes)} routes, Coût: {self.cost:.2f}, Faisable: {self.is_feasible}"print("Classe VRPTWSolution définie!")

In [ ]:
def calculate_distance_matrix(instance):    """Calculer la matrice de distances euclidiennes"""    coords = instance['node_coord']    n = len(coords)    dist_matrix = np.zeros((n, n))        for i in range(n):        for j in range(n):            if i != j:                dx = coords[i][0] - coords[j][0]                dy = coords[i][1] - coords[j][1]                dist_matrix[i][j] = np.sqrt(dx*dx + dy*dy)        return dist_matrixdef calculate_route_cost_and_time(route, instance, dist_matrix):    """    Calculer le coût et la faisabilité d'une route    Retourne: (coût, charge, violations_temps, est_faisable)    """    if len(route) == 0:        return 0.0, 0, 0, True        depot = 0    cost = 0.0    load = 0    current_time = 0.0    time_violations = 0        # Du dépôt au premier client    prev = depot    for customer in route:        # Distance        cost += dist_matrix[prev][customer]                # Temps de trajet        current_time += dist_matrix[prev][customer]                # Fenêtre temporelle        ready_time = instance['time_window'][customer][0]        due_date = instance['time_window'][customer][1]        service_time = instance['service_time'][customer]                # Attendre si on arrive trop tôt        if current_time < ready_time:            current_time = ready_time                # Vérifier si on arrive trop tard        if current_time > due_date:            time_violations += 1                # Ajouter le temps de service        current_time += service_time                # Charge        load += instance['demand'][customer]                prev = customer        # Retour au dépôt    cost += dist_matrix[prev][depot]    current_time += dist_matrix[prev][depot]        # Vérifier la fenêtre temporelle du dépôt au retour    depot_due = instance['time_window'][depot][1]    if current_time > depot_due:        time_violations += 1        # Vérifier la capacité    capacity = instance['capacity']    is_feasible = (load <= capacity) and (time_violations == 0)        return cost, load, time_violations, is_feasibledef evaluate_solution(solution):    """Évaluer complètement une solution"""    instance = solution.instance    dist_matrix = calculate_distance_matrix(instance)        total_cost = 0.0    total_capacity_violations = 0    total_time_violations = 0    is_feasible = True        for route in solution.routes:        cost, load, time_viol, route_feasible = calculate_route_cost_and_time(            route, instance, dist_matrix        )        total_cost += cost                if load > instance['capacity']:            total_capacity_violations += 1            is_feasible = False                total_time_violations += time_viol        if time_viol > 0:            is_feasible = False        # Ajouter des pénalités pour les violations    if total_capacity_violations > 0:        total_cost += config['constraints']['capacity_penalty'] * total_capacity_violations    if total_time_violations > 0:        total_cost += config['constraints']['time_window_penalty'] * total_time_violations        solution.cost = total_cost    solution.is_feasible = is_feasible    solution.capacity_violations = total_capacity_violations    solution.time_violations = total_time_violations        return solutionprint("Fonctions de calcul de distance et de temps définies!")

## Algorithme de Clark-Wright pour la construction de solution initiale

In [ ]:
def clark_wright_savings(instance, config):    """    Construire une solution initiale avec l'heuristique de Clark-Wright    adaptée pour le VRPTW    """    n_customers = len(instance['demand']) - 1  # Exclure le dépôt    depot = 0    dist_matrix = calculate_distance_matrix(instance)    lambda_param = config['clark_wright']['lambda_parameter']    randomness = config['clark_wright']['randomness']        # Calculer les économies pour toutes les paires de clients    savings = []    for i in range(1, n_customers + 1):        for j in range(i + 1, n_customers + 1):            # Économie classique de Clark-Wright            saving = (dist_matrix[depot][i] + dist_matrix[depot][j] -                      lambda_param * dist_matrix[i][j])            savings.append((saving, i, j))        # Trier les économies par ordre décroissant    savings.sort(reverse=True, key=lambda x: x[0])        # Initialiser: chaque client sur sa propre route    routes = [[i] for i in range(1, n_customers + 1)]    route_of_customer = {i: i-1 for i in range(1, n_customers + 1)}        # Fusionner les routes en utilisant les économies    for saving, i, j in savings:        # Ajouter un peu d'aléatoire        if random.random() > randomness:            route_i = route_of_customer.get(i, -1)            route_j = route_of_customer.get(j, -1)                        if route_i == -1 or route_j == -1 or route_i == route_j:                continue                        # Vérifier si i et j sont aux extrémités de leurs routes            if routes[route_i][-1] == i and routes[route_j][0] == j:                # Fusionner route_i et route_j                merged = routes[route_i] + routes[route_j]            elif routes[route_i][0] == i and routes[route_j][-1] == j:                # Fusionner route_j et route_i                merged = routes[route_j] + routes[route_i]            elif routes[route_i][-1] == i and routes[route_j][-1] == j:                # Inverser route_j et fusionner                merged = routes[route_i] + routes[route_j][::-1]            elif routes[route_i][0] == i and routes[route_j][0] == j:                # Inverser route_i et fusionner                merged = routes[route_i][::-1] + routes[route_j]            else:                continue                        # Vérifier la faisabilité de la route fusionnée            cost, load, time_viol, is_feasible = calculate_route_cost_and_time(                merged, instance, dist_matrix            )                        if is_feasible:                # Mettre à jour les routes                new_route_idx = route_i                old_route_idx = route_j                                routes[new_route_idx] = merged                routes[old_route_idx] = []                                # Mettre à jour le dictionnaire                for customer in merged:                    route_of_customer[customer] = new_route_idx        # Supprimer les routes vides    routes = [r for r in routes if len(r) > 0]        # Créer l'objet solution    solution = VRPTWSolution(instance)    solution.routes = routes    solution = evaluate_solution(solution)        return solutionprint("Algorithme de Clark-Wright défini!")

## Opérateurs VND (Variable Neighborhood Descent)

In [ ]:
def relocate_operator(solution):    """    Opérateur de relocalisation: déplacer un client d'une route à une autre    """    instance = solution.instance    dist_matrix = calculate_distance_matrix(instance)    best_solution = solution.copy()    improved = False        for route_idx in range(len(solution.routes)):        for pos in range(len(solution.routes[route_idx])):            customer = solution.routes[route_idx][pos]                        # Essayer de déplacer vers toutes les autres routes            for other_route_idx in range(len(solution.routes)):                if route_idx == other_route_idx:                    continue                                # Essayer toutes les positions dans l'autre route                for insert_pos in range(len(solution.routes[other_route_idx]) + 1):                    # Créer une nouvelle solution                    new_solution = solution.copy()                                        # Retirer le client de la route originale                    new_solution.routes[route_idx].pop(pos)                                        # Insérer dans la nouvelle route                    new_solution.routes[other_route_idx].insert(insert_pos, customer)                                        # Évaluer                    new_solution = evaluate_solution(new_solution)                                        if new_solution.cost < best_solution.cost:                        best_solution = new_solution.copy()                        improved = True        best_solution.remove_empty_routes()    return best_solution, improvedprint("Opérateur de relocalisation défini!")

In [ ]:
def swap_operator(solution):    """    Opérateur d'échange: échanger deux clients entre deux routes différentes    """    instance = solution.instance    dist_matrix = calculate_distance_matrix(instance)    best_solution = solution.copy()    improved = False        for route1_idx in range(len(solution.routes)):        for pos1 in range(len(solution.routes[route1_idx])):            for route2_idx in range(route1_idx + 1, len(solution.routes)):                for pos2 in range(len(solution.routes[route2_idx])):                    # Créer une nouvelle solution                    new_solution = solution.copy()                                        # Échanger les clients                    customer1 = new_solution.routes[route1_idx][pos1]                    customer2 = new_solution.routes[route2_idx][pos2]                                        new_solution.routes[route1_idx][pos1] = customer2                    new_solution.routes[route2_idx][pos2] = customer1                                        # Évaluer                    new_solution = evaluate_solution(new_solution)                                        if new_solution.cost < best_solution.cost:                        best_solution = new_solution.copy()                        improved = True        return best_solution, improvedprint("Opérateur d'échange défini!")

In [ ]:
def two_opt_operator(solution):    """    Opérateur 2-opt: optimisation intra-route    """    instance = solution.instance    dist_matrix = calculate_distance_matrix(instance)    best_solution = solution.copy()    improved = False        for route_idx in range(len(solution.routes)):        route = solution.routes[route_idx]        n = len(route)                if n < 2:            continue                for i in range(n - 1):            for j in range(i + 1, n):                # Créer une nouvelle solution                new_solution = solution.copy()                                # Inverser le segment [i+1, j]                new_solution.routes[route_idx] = (                    route[:i+1] + route[i+1:j+1][::-1] + route[j+1:]                )                                # Évaluer                new_solution = evaluate_solution(new_solution)                                if new_solution.cost < best_solution.cost:                    best_solution = new_solution.copy()                    improved = True        return best_solution, improvedprint("Opérateur 2-opt défini!")

In [ ]:
def variable_neighborhood_descent(solution, config):    """    Variable Neighborhood Descent: recherche locale systématique    """    operators = {        'relocate': relocate_operator,        'swap': swap_operator,        'two_opt': two_opt_operator    }        neighborhoods = config['vnd']['neighborhoods']    max_no_improve = config['vnd']['max_iterations_without_improvement']        current_solution = solution.copy()    iterations_no_improve = 0        while iterations_no_improve < max_no_improve:        improved_in_vnd = False                for neighborhood in neighborhoods:            if neighborhood in operators:                new_solution, improved = operators[neighborhood](current_solution)                                if improved:                    current_solution = new_solution                    improved_in_vnd = True                    iterations_no_improve = 0                    break                if not improved_in_vnd:            iterations_no_improve += 1        return current_solutionprint("VND défini!")

## Opérateurs de destruction ALNS

In [ ]:
def random_removal(solution, num_remove):    """Suppression aléatoire de clients"""    new_solution = solution.copy()    all_customers = new_solution.get_all_customers()        if len(all_customers) < num_remove:        num_remove = len(all_customers)        customers_to_remove = random.sample(all_customers, num_remove)    removed_customers = []        for customer in customers_to_remove:        for route in new_solution.routes:            if customer in route:                route.remove(customer)                removed_customers.append(customer)                break        new_solution.remove_empty_routes()    return new_solution, removed_customersprint("Opérateur de suppression aléatoire défini!")

In [ ]:
def worst_removal(solution, num_remove):    """Suppression des clients les plus coûteux"""    instance = solution.instance    dist_matrix = calculate_distance_matrix(instance)    depot = 0        new_solution = solution.copy()    customer_costs = []        # Calculer le coût de chaque client dans sa route    for route in new_solution.routes:        for i, customer in enumerate(route):            # Coût = distance avant + distance après - distance directe            prev = route[i-1] if i > 0 else depot            next_c = route[i+1] if i < len(route)-1 else depot                        cost_with = dist_matrix[prev][customer] + dist_matrix[customer][next_c]            cost_without = dist_matrix[prev][next_c]            customer_cost = cost_with - cost_without                        customer_costs.append((customer_cost, customer))        # Trier par coût décroissant    customer_costs.sort(reverse=True)        # Supprimer les N plus coûteux    removed_customers = []    for i in range(min(num_remove, len(customer_costs))):        customer = customer_costs[i][1]        for route in new_solution.routes:            if customer in route:                route.remove(customer)                removed_customers.append(customer)                break        new_solution.remove_empty_routes()    return new_solution, removed_customersprint("Opérateur de suppression du pire défini!")

In [ ]:
def shaw_removal(solution, num_remove):    """    Suppression de clients similaires (proximité spatiale et temporelle)    """    instance = solution.instance    dist_matrix = calculate_distance_matrix(instance)        new_solution = solution.copy()    all_customers = new_solution.get_all_customers()        if len(all_customers) == 0:        return new_solution, []        # Sélectionner un client aléatoire comme graine    seed_customer = random.choice(all_customers)    removed_customers = [seed_customer]        # Calculer la similarité avec tous les autres clients    similarities = []    for customer in all_customers:        if customer != seed_customer:            # Similarité basée sur la distance et la fenêtre temporelle            dist = dist_matrix[seed_customer][customer]            time_diff = abs(instance['time_window'][seed_customer][0] -                           instance['time_window'][customer][0])            similarity = dist + 0.1 * time_diff  # Pondération            similarities.append((similarity, customer))        # Trier par similarité croissante    similarities.sort()        # Supprimer les clients les plus similaires    for i in range(min(num_remove - 1, len(similarities))):        removed_customers.append(similarities[i][1])        # Supprimer de la solution    for customer in removed_customers:        for route in new_solution.routes:            if customer in route:                route.remove(customer)                break        new_solution.remove_empty_routes()    return new_solution, removed_customersprint("Opérateur de suppression de Shaw défini!")

In [ ]:
def route_removal(solution, num_remove):    """Suppression d'une route complète"""    new_solution = solution.copy()        if len(new_solution.routes) == 0:        return new_solution, []        # Sélectionner une route aléatoire    route_idx = random.randint(0, len(new_solution.routes) - 1)    removed_customers = new_solution.routes[route_idx].copy()    new_solution.routes.pop(route_idx)        return new_solution, removed_customersprint("Opérateur de suppression de route défini!")

## Opérateurs de réparation ALNS

In [ ]:
def greedy_insertion(solution, removed_customers):    """Insertion gloutonne des clients supprimés"""    instance = solution.instance    dist_matrix = calculate_distance_matrix(instance)        new_solution = solution.copy()        for customer in removed_customers:        best_cost = float('inf')        best_route_idx = -1        best_pos = -1                # Essayer d'insérer dans toutes les routes existantes        for route_idx in range(len(new_solution.routes)):            for pos in range(len(new_solution.routes[route_idx]) + 1):                # Insérer temporairement                test_route = new_solution.routes[route_idx].copy()                test_route.insert(pos, customer)                                # Calculer le coût                cost, load, time_viol, is_feasible = calculate_route_cost_and_time(                    test_route, instance, dist_matrix                )                                if is_feasible and cost < best_cost:                    best_cost = cost                    best_route_idx = route_idx                    best_pos = pos                # Essayer de créer une nouvelle route        new_route_cost, load, time_viol, is_feasible = calculate_route_cost_and_time(            [customer], instance, dist_matrix        )                if is_feasible and new_route_cost < best_cost:            new_solution.routes.append([customer])        elif best_route_idx != -1:            new_solution.routes[best_route_idx].insert(best_pos, customer)        else:            # Si aucune insertion faisable, créer une nouvelle route quand même            new_solution.routes.append([customer])        new_solution = evaluate_solution(new_solution)    return new_solutionprint("Opérateur d'insertion gloutonne défini!")

In [ ]:
def regret_insertion(solution, removed_customers):    """    Insertion basée sur le regret:     insérer d'abord les clients qui ont le plus grand regret    """    instance = solution.instance    dist_matrix = calculate_distance_matrix(instance)        new_solution = solution.copy()    remaining_customers = removed_customers.copy()        while remaining_customers:        max_regret = -float('inf')        best_customer = None        best_route_idx = -1        best_pos = -1                for customer in remaining_customers:            # Trouver les 2 meilleures positions pour ce client            insertion_costs = []                        for route_idx in range(len(new_solution.routes)):                for pos in range(len(new_solution.routes[route_idx]) + 1):                    test_route = new_solution.routes[route_idx].copy()                    test_route.insert(pos, customer)                                        cost, load, time_viol, is_feasible = calculate_route_cost_and_time(                        test_route, instance, dist_matrix                    )                                        if is_feasible:                        insertion_costs.append((cost, route_idx, pos))                        # Ajouter l'option de créer une nouvelle route            new_route_cost, load, time_viol, is_feasible = calculate_route_cost_and_time(                [customer], instance, dist_matrix            )            if is_feasible:                insertion_costs.append((new_route_cost, -1, -1))                        # Calculer le regret            if len(insertion_costs) >= 2:                insertion_costs.sort()                regret = insertion_costs[1][0] - insertion_costs[0][0]                                if regret > max_regret:                    max_regret = regret                    best_customer = customer                    best_route_idx = insertion_costs[0][1]                    best_pos = insertion_costs[0][2]                # Insérer le client avec le plus grand regret        if best_customer is not None:            if best_route_idx == -1:                new_solution.routes.append([best_customer])            else:                new_solution.routes[best_route_idx].insert(best_pos, best_customer)            remaining_customers.remove(best_customer)        else:            # Si aucune insertion faisable, créer de nouvelles routes            for customer in remaining_customers:                new_solution.routes.append([customer])            break        new_solution = evaluate_solution(new_solution)    return new_solutionprint("Opérateur d'insertion par regret défini!")

In [ ]:
def best_insertion(solution, removed_customers):    """Meilleure insertion: insérer chaque client à sa meilleure position"""    instance = solution.instance    dist_matrix = calculate_distance_matrix(instance)        new_solution = solution.copy()        # Trier les clients par ordre de difficulté d'insertion    # (clients avec fenêtres temporelles étroites en premier)    customers_sorted = sorted(        removed_customers,        key=lambda c: instance['time_window'][c][1] - instance['time_window'][c][0]    )        for customer in customers_sorted:        best_cost_increase = float('inf')        best_route_idx = -1        best_pos = -1        current_cost = new_solution.cost if new_solution.cost != float('inf') else 0                # Essayer toutes les positions        for route_idx in range(len(new_solution.routes)):            original_cost, _, _, _ = calculate_route_cost_and_time(                new_solution.routes[route_idx], instance, dist_matrix            )                        for pos in range(len(new_solution.routes[route_idx]) + 1):                test_route = new_solution.routes[route_idx].copy()                test_route.insert(pos, customer)                                new_cost, load, time_viol, is_feasible = calculate_route_cost_and_time(                    test_route, instance, dist_matrix                )                                if is_feasible:                    cost_increase = new_cost - original_cost                    if cost_increase < best_cost_increase:                        best_cost_increase = cost_increase                        best_route_idx = route_idx                        best_pos = pos                # Nouvelle route        new_route_cost, load, time_viol, is_feasible = calculate_route_cost_and_time(            [customer], instance, dist_matrix        )                if is_feasible and new_route_cost < best_cost_increase:            new_solution.routes.append([customer])        elif best_route_idx != -1:            new_solution.routes[best_route_idx].insert(best_pos, customer)        else:            new_solution.routes.append([customer])        new_solution = evaluate_solution(new_solution)    return new_solutionprint("Opérateur de meilleure insertion défini!")

## Algorithme principal ALNS

In [ ]:
def adaptive_large_neighborhood_search(initial_solution, config):    """    Adaptive Large Neighborhood Search pour VRPTW    """    # Paramètres    max_iterations = config['alns']['max_iterations']    segment_iterations = config['alns']['segment_iterations']    destroy_min = config['alns']['destroy_percentage_min']    destroy_max = config['alns']['destroy_percentage_max']    initial_temp = config['alns']['initial_temperature']    cooling_rate = config['alns']['cooling_rate']        # Scores    score_best = config['alns']['weights']['score_best_solution']    score_better = config['alns']['weights']['score_better_solution']    score_accepted = config['alns']['weights']['score_accepted_solution']        # Adaptation    reaction = config['alns']['adaptation']['reaction_factor']    decay = config['alns']['adaptation']['decay_parameter']        # Opérateurs    destroy_ops = {        'random_removal': random_removal,        'worst_removal': worst_removal,        'shaw_removal': shaw_removal,        'route_removal': route_removal    }        repair_ops = {        'greedy_insertion': greedy_insertion,        'regret_insertion': regret_insertion,        'best_insertion': best_insertion    }        # Initialiser les poids des opérateurs    destroy_weights = {name: 1.0 for name in destroy_ops.keys()}    repair_weights = {name: 1.0 for name in repair_ops.keys()}        # Initialiser les scores    destroy_scores = {name: 0.0 for name in destroy_ops.keys()}    repair_scores = {name: 0.0 for name in repair_ops.keys()}    destroy_uses = {name: 0 for name in destroy_ops.keys()}    repair_uses = {name: 0 for name in repair_ops.keys()}        # Solutions    current_solution = initial_solution.copy()    best_solution = initial_solution.copy()        temperature = initial_temp        # Historique de convergence    convergence_history = []        # ALNS loop    for iteration in range(max_iterations):        # Nombre de clients à supprimer        n_customers = len(current_solution.get_all_customers())        num_remove = int(n_customers * random.uniform(destroy_min, destroy_max))        num_remove = max(1, min(num_remove, n_customers))                # Sélectionner un opérateur de destruction (roulette wheel)        total_weight = sum(destroy_weights.values())        rand = random.uniform(0, total_weight)        cumsum = 0        selected_destroy = None        for name, weight in destroy_weights.items():            cumsum += weight            if rand <= cumsum:                selected_destroy = name                break                # Sélectionner un opérateur de réparation        total_weight = sum(repair_weights.values())        rand = random.uniform(0, total_weight)        cumsum = 0        selected_repair = None        for name, weight in repair_weights.items():            cumsum += weight            if rand <= cumsum:                selected_repair = name                break                # Appliquer les opérateurs        destroyed_solution, removed_customers = destroy_ops[selected_destroy](            current_solution, num_remove        )        new_solution = repair_ops[selected_repair](destroyed_solution, removed_customers)                # Mettre à jour les compteurs        destroy_uses[selected_destroy] += 1        repair_uses[selected_repair] += 1                # Critère d'acceptation (Simulated Annealing)        delta = new_solution.cost - current_solution.cost        accept = False        score = 0                if new_solution.cost < best_solution.cost:            # Nouvelle meilleure solution            best_solution = new_solution.copy()            current_solution = new_solution.copy()            accept = True            score = score_best        elif new_solution.cost < current_solution.cost:            # Solution améliorante            current_solution = new_solution.copy()            accept = True            score = score_better        elif random.random() < math.exp(-delta / temperature):            # Acceptation probabiliste            current_solution = new_solution.copy()            accept = True            score = score_accepted                # Mettre à jour les scores        if accept:            destroy_scores[selected_destroy] += score            repair_scores[selected_repair] += score                # Refroidissement        temperature *= cooling_rate                # Adaptation des poids        if (iteration + 1) % segment_iterations == 0:            # Mettre à jour les poids            for name in destroy_ops.keys():                if destroy_uses[name] > 0:                    avg_score = destroy_scores[name] / destroy_uses[name]                    destroy_weights[name] = (destroy_weights[name] * (1 - reaction) +                                            reaction * avg_score)                        for name in repair_ops.keys():                if repair_uses[name] > 0:                    avg_score = repair_scores[name] / repair_uses[name]                    repair_weights[name] = (repair_weights[name] * (1 - reaction) +                                           reaction * avg_score)                        # Décroissance des scores            for name in destroy_ops.keys():                destroy_scores[name] *= decay            for name in repair_ops.keys():                repair_scores[name] *= decay                # Enregistrer la convergence        convergence_history.append(best_solution.cost)                # Affichage de la progression        if config['general']['verbose'] and (iteration + 1) % 100 == 0:            print(f"Itération {iteration+1}/{max_iterations}: "                  f"Meilleure = {best_solution.cost:.2f}, "                  f"Courante = {current_solution.cost:.2f}, "                  f"Temp = {temperature:.2f}")                # Appliquer VND périodiquement        if (iteration + 1) % config['general']['apply_vnd_frequency'] == 0:            current_solution = variable_neighborhood_descent(current_solution, config)            if current_solution.cost < best_solution.cost:                best_solution = current_solution.copy()        return best_solution, convergence_historyprint("Algorithme ALNS défini!")

## Visualisation des solutions

In [ ]:
def plot_vrptw_solution(solution, instance, title="Solution VRPTW"):    """Visualiser une solution VRPTW"""    fig, ax = plt.subplots(figsize=tuple(config['visualization']['plot_figsize']))        coords = instance['node_coord']    depot = coords[0]        # Couleurs pour les routes    colors = plt.cm.tab20(np.linspace(0, 1, len(solution.routes)))        # Dessiner le dépôt    ax.plot(depot[0], depot[1], 'ks', markersize=15, label='Dépôt', zorder=3)        # Dessiner chaque route    for idx, (route, color) in enumerate(zip(solution.routes, colors)):        route_coords = [depot] + [coords[c] for c in route] + [depot]        xs = [c[0] for c in route_coords]        ys = [c[1] for c in route_coords]                ax.plot(xs, ys, 'o-', color=color, linewidth=2,                 label=f'Route {idx+1}', markersize=8, alpha=0.7)        # Dessiner les clients    customer_coords = coords[1:]    ax.plot([c[0] for c in customer_coords], [c[1] for c in customer_coords],             'o', color='lightblue', markersize=6, zorder=1)        ax.set_xlabel('Coordonnée X', fontsize=12)    ax.set_ylabel('Coordonnée Y', fontsize=12)    ax.set_title(f'{title}\nCoût: {solution.cost:.2f}, Routes: {len(solution.routes)}',                fontsize=14, fontweight='bold')    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)    ax.grid(True, alpha=0.3)        plt.tight_layout()    return figdef plot_convergence(convergence_history, title="Convergence ALNS"):    """Visualiser la convergence de l'algorithme"""    fig, ax = plt.subplots(figsize=tuple(config['visualization']['plot_figsize']))        ax.plot(convergence_history, linewidth=2, color='blue')    ax.set_xlabel('Itération', fontsize=12)    ax.set_ylabel('Coût de la meilleure solution', fontsize=12)    ax.set_title(title, fontsize=14, fontweight='bold')    ax.grid(True, alpha=0.3)        plt.tight_layout()    return figprint("Fonctions de visualisation définies!")

## Résolution des instances Solomon

In [ ]:
def solve_vrptw_instance(instance_path, config):    """Résoudre une instance VRPTW"""    print(f"\n{'='*80}")    print(f"Résolution de l'instance: {os.path.basename(instance_path)}")    print(f"{'='*80}")        # Lire l'instance    instance = vrplib.read_instance(instance_path, instance_format="solomon")    print(f"Instance chargée: {instance['name']}")    print(f"Nombre de clients: {len(instance['demand']) - 1}")    print(f"Capacité des véhicules: {instance['capacity']}")        # Temps de début    start_time = time.time()        # Construction initiale avec Clark-Wright    print("\nConstruction de la solution initiale avec Clark-Wright...")    initial_solution = clark_wright_savings(instance, config)    print(f"Solution initiale: Coût = {initial_solution.cost:.2f}, "          f"Routes = {len(initial_solution.routes)}, "          f"Faisable = {initial_solution.is_feasible}")        # Amélioration avec VND    print("\nAmélioration avec VND...")    vnd_solution = variable_neighborhood_descent(initial_solution, config)    print(f"Après VND: Coût = {vnd_solution.cost:.2f}, "          f"Routes = {len(vnd_solution.routes)}")        # Optimisation avec ALNS    print("\nOptimisation avec ALNS...")    final_solution, convergence = adaptive_large_neighborhood_search(vnd_solution, config)        # Temps de fin    end_time = time.time()    computation_time = end_time - start_time        print(f"\nSolution finale: Coût = {final_solution.cost:.2f}, "          f"Routes = {len(final_solution.routes)}, "          f"Faisable = {final_solution.is_feasible}")    print(f"Temps de calcul: {computation_time:.2f} secondes")        return {        'instance_name': instance['name'],        'initial_solution': initial_solution,        'vnd_solution': vnd_solution,        'final_solution': final_solution,        'convergence': convergence,        'computation_time': computation_time,        'instance': instance    }print("Fonction de résolution définie!")

In [ ]:
# Créer le dossier de sortieoutput_dir = 'solutions_TW_ALNS'os.makedirs(output_dir, exist_ok=True)os.makedirs(os.path.join(output_dir, 'plots'), exist_ok=True)os.makedirs(os.path.join(output_dir, 'csv'), exist_ok=True)print(f"Dossier de sortie créé: {output_dir}")# Trouver les instances Solomonsolomon_instances = []data_dirs = [    'data/cvrplib/Vrp-Set-Solomon',    'data']for data_dir in data_dirs:    if os.path.exists(data_dir):        for file in os.listdir(data_dir):            if file.endswith('.txt') and (file.startswith('C') or                                          file.startswith('R') or                                          file.startswith('RC')):                solomon_instances.append(os.path.join(data_dir, file))# Limiter le nombre d'instances pour le testsolomon_instances = sorted(list(set(solomon_instances)))[:5]print(f"\nInstances trouvées: {len(solomon_instances)}")for inst in solomon_instances:    print(f"  - {os.path.basename(inst)}")

In [ ]:
# Résoudre toutes les instancesresults = []for instance_path in solomon_instances:    try:        result = solve_vrptw_instance(instance_path, config)        results.append(result)                # Sauvegarder la visualisation        if config['visualization']['plot_routes']:            fig = plot_vrptw_solution(                result['final_solution'],                result['instance'],                title=f"Solution VRPTW - {result['instance_name']}"            )            plot_path = os.path.join(output_dir, 'plots',                                     f"{result['instance_name']}_solution.png")            fig.savefig(plot_path, dpi=config['visualization']['plot_dpi'],                        bbox_inches='tight')            plt.close(fig)            print(f"Graphique sauvegardé: {plot_path}")                # Sauvegarder la convergence        if config['visualization']['plot_convergence']:            fig = plot_convergence(                result['convergence'],                title=f"Convergence ALNS - {result['instance_name']}"            )            plot_path = os.path.join(output_dir, 'plots',                                     f"{result['instance_name']}_convergence.png")            fig.savefig(plot_path, dpi=config['visualization']['plot_dpi'],                        bbox_inches='tight')            plt.close(fig)            print(f"Graphique de convergence sauvegardé: {plot_path}")                # Sauvegarder la solution        solution_path = os.path.join(output_dir, f"{result['instance_name']}_alns.sol")        with open(solution_path, 'w') as f:            for idx, route in enumerate(result['final_solution'].routes, 1):                f.write(f"Route #{idx}: {' '.join(map(str, route))}\n")            f.write(f"Cost {result['final_solution'].cost:.2f}\n")        print(f"Solution sauvegardée: {solution_path}")            except Exception as e:        print(f"Erreur lors de la résolution de {instance_path}: {e}")        import traceback        traceback.print_exc()print(f"\n{'='*80}")print(f"Résolution terminée! {len(results)} instances résolues avec succès.")print(f"{'='*80}")

In [ ]:
# Créer un résumé des résultatssummary_data = []for result in results:    summary_data.append({        'Instance': result['instance_name'],        'Coût Initial (Clark-Wright)': f"{result['initial_solution'].cost:.2f}",        'Coût après VND': f"{result['vnd_solution'].cost:.2f}",        'Coût Final (ALNS)': f"{result['final_solution'].cost:.2f}",        'Nombre de Routes': len(result['final_solution'].routes),        'Faisable': 'Oui' if result['final_solution'].is_feasible else 'Non',        'Temps (s)': f"{result['computation_time']:.2f}",        'Amélioration (%)': f"{((result['initial_solution'].cost - result['final_solution'].cost) / result['initial_solution'].cost * 100):.2f}"    })# Créer un DataFramesummary_df = pd.DataFrame(summary_data)# Afficher le résuméprint("\n" + "="*120)print("RÉSUMÉ DES RÉSULTATS")print("="*120)print(summary_df.to_string(index=False))print("="*120)# Sauvegarder en CSVcsv_path = os.path.join(output_dir, 'csv', 'summary_results.csv')summary_df.to_csv(csv_path, index=False, encoding='utf-8')print(f"\nRésumé sauvegardé dans: {csv_path}")

## RésultatsLes résultats ont été sauvegardés dans le dossier `solutions_TW_ALNS/`:- **Fichiers de solution** (.sol): Solutions complètes au format VRPLIB- **Graphiques** (plots/): Visualisations des routes et convergence- **Fichiers CSV** (csv/): Résumé des résultats en format tabulaire### Prochaines étapes:1. Analyser les graphiques de convergence pour ajuster les paramètres ALNS2. Comparer avec les meilleures solutions connues si disponibles3. Ajuster les paramètres dans `config_ALNS.yaml` pour améliorer la qualité4. Tester sur d'autres instances Solomon (R, RC, etc.)### Notes sur l'algorithme:- **Clark-Wright**: Heuristique classique adaptée pour VRPTW- **VND**: Amélioration locale déterministe avec 3 opérateurs- **ALNS**: Recherche adaptative avec 4 opérateurs de destruction et 3 de réparation- Les poids des opérateurs s'adaptent dynamiquement selon leur performance**Bon travail avec le solveur VRPTW-ALNS! 🚛📦**